Goal for this version (2.0, 2019-01-19): document process of downloading full year of rolling stone album charts and the accompanying spotify information.

# Make Dataset: raw data

## note: need to process the rolling stone data somewhat before accessing the raw spotify data

## Setup

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import dotenv
import os
import glob

import sys

import pprint

import matplotlib.pyplot as plt

import typing
import pandas as pd
import time

In [2]:
from bs4 import BeautifulSoup

In [3]:
project_dir = str(os.path.dirname(os.path.abspath('')))
sys.path.append(project_dir)

from src.data import scrape_rolling_stone as scrapeRS # import local scripts

from src.data.pickle_util import save_pickle, load_pickle

from src.data import get_spotify_api_data as gsad

## download rolling stone website to canned soup text files

In [5]:
if False: # make me True to download
    start_date = '2019-01-03'
    end_date = '2020-01-18'
    scrapeRS.get_multiple_rs_200_albums_pages(start_date, end_date)
    print('download complete')
else:
    print('skipped download')

skipped download


## Load all rolling stone website soup objects from canned soup text files

find filenames

In [6]:
if False: # make me true if needed
    raw_data_folder = project_dir + '/data/raw/'
    os.chdir(raw_data_folder)
    canned_soup_file_names = []
    for file_name in glob.glob("canned_soup_of_rs_200_albums*.txt"):
        canned_soup_file_names.append(file_name)
    # print(canned_soup_files)

load soups, using filenames list. Also create soup objects from these files

In [9]:
if False: # make me true to load soups
    soups = []
    for file_name in canned_soup_file_names:
        file_path = project_dir + '/data/raw/' + file_name
        with open(file_path) as f:
            soup = BeautifulSoup(f)
        soups.append(soup)
    print('loaded canned soup and created soup objects')

loaded canned soup and created soup objects


## Create list of dataframes from soup objects

In [12]:
if False: # make me true if needed
    rs_200_dfs_list = []
    for soup in soups:
        df = scrapeRS.make_rs_200_df(soup)
        rs_200_dfs_list.append(df)
    print('completed making dataframes from soup objects')

completed making dataframes from soup objects


In [13]:
sys.getsizeof(rs_200_dfs_list) # returns object size in bytes

536

## Save list of dataframes as a pickle

In [14]:
if False: # make me True to save list of dateframes
    start_date = '2019-01-03'
    end_date = '2020-01-18'
    base_path = project_dir + '/data/interim/'
    file_name = 'rs_200_dfs_list' + start_date + "_to_" + end_date + '.pickle'
    file_path = base_path + file_name
    save_pickle(rs_200_dfs_list, file_path)

AssertionError: file already exists, write operation canceled

## Load list of dataframes from pickle

In [15]:
if False: # make me false to not load list of dataframes from pickle
    start_date = '2019-01-03'
    end_date = '2020-01-18'
    base_path = project_dir + '/data/interim/'
    file_name = 'rs_200_dfs_list' + start_date + "_to_" + end_date + '.pickle'
    file_path = base_path + file_name
rs_200_dfs_list = load_pickle(file_path)

## Make concatenated dataframe from the list of dataframes

In [36]:
if True: # make me True if needed
    rs_200_year_df = pd.concat(rs_200_dfs_list).reset_index(drop=True)

In [37]:
rs_200_year_df

,rank,title,caption_artist,album_units,record_label
0,1,Indigo,Chris Brown,110.4K,RCA Records
1,2,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,50.9K,Interscope
2,3,Let's Rock,The Black Keys,47.6K,Nonesuch Records
3,4,7 EP,Lil Nas X,42.8K,Columbia Records
4,5,Free Spirit,Khalid,37.3K,RCA Records
...,...,...,...,...,...
10990,196,Artist,A Boogie wit da Hoodie,3.6K,Atlantic Records
10991,197,Different 'Round Here,Riley Green,3.5K,BMLG Records
10992,198,Culture,Migos,3.5K,300 Entertainment
10993,199,Night Visions,Imagine Dragons,3.5K,Interscope


## Save csv of concatenated dataframe to interim data

In [38]:
directory = project_dir + '/data/interim/'
file_name = 'rs_200_albums_year_2019.csv'
file_path = directory + file_name
if True: # make me true to save csv of dataframe
    rs_200_year_df.to_csv(file_path, index=False)

In [31]:
## load csv of dataframe with all rolling stone albums from the year

In [39]:
rs_200_year_df = pd.read_csv(file_path)

In [40]:
rs_200_year_df

,rank,title,caption_artist,album_units,record_label
0,1,Indigo,Chris Brown,110.4K,RCA Records
1,2,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,50.9K,Interscope
2,3,Let's Rock,The Black Keys,47.6K,Nonesuch Records
3,4,7 EP,Lil Nas X,42.8K,Columbia Records
4,5,Free Spirit,Khalid,37.3K,RCA Records
...,...,...,...,...,...
10990,196,Artist,A Boogie wit da Hoodie,3.6K,Atlantic Records
10991,197,Different 'Round Here,Riley Green,3.5K,BMLG Records
10992,198,Culture,Migos,3.5K,300 Entertainment
10993,199,Night Visions,Imagine Dragons,3.5K,Interscope


## Setup rs 200 albums data for getting spotify data

Clean album titles to make the spotify search work better

In [42]:
rs_200_year_df['title_cleaned'] = gsad.clean_rs_album_titles_2020_01_09(rs_200_year_df['title'].copy())

inspect the result

In [43]:
rs_200_year_df

,rank,title,caption_artist,album_units,record_label,title_cleaned
0,1,Indigo,Chris Brown,110.4K,RCA Records,Indigo
1,2,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,50.9K,Interscope,"When We All Fall Asleep, Where Do We Go?"
2,3,Let's Rock,The Black Keys,47.6K,Nonesuch Records,Let's Rock
3,4,7 EP,Lil Nas X,42.8K,Columbia Records,7 EP
4,5,Free Spirit,Khalid,37.3K,RCA Records,Free Spirit
...,...,...,...,...,...,...
10990,196,Artist,A Boogie wit da Hoodie,3.6K,Atlantic Records,Artist
10991,197,Different 'Round Here,Riley Green,3.5K,BMLG Records,Different 'Round Here
10992,198,Culture,Migos,3.5K,300 Entertainment,Culture
10993,199,Night Visions,Imagine Dragons,3.5K,Interscope,Night Visions


save to csv and load from csv

In [47]:
directory = project_dir + '/data/interim/'
file_name = 'rs_200_albums_year_2019_cleaned.csv'
file_path = directory + file_name
if False: # make me true to save csv of dataframe
    rs_200_year_df.to_csv(file_path, index=False)
if True: # make me True if needed
    rs_200_year_df = pd.read_csv(file_path)

In [48]:
rs_200_year_df

,rank,title,caption_artist,album_units,record_label,title_cleaned
0,1,Indigo,Chris Brown,110.4K,RCA Records,Indigo
1,2,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,50.9K,Interscope,"When We All Fall Asleep, Where Do We Go?"
2,3,Let's Rock,The Black Keys,47.6K,Nonesuch Records,Let's Rock
3,4,7 EP,Lil Nas X,42.8K,Columbia Records,7 EP
4,5,Free Spirit,Khalid,37.3K,RCA Records,Free Spirit
...,...,...,...,...,...,...
10990,196,Artist,A Boogie wit da Hoodie,3.6K,Atlantic Records,Artist
10991,197,Different 'Round Here,Riley Green,3.5K,BMLG Records,Different 'Round Here
10992,198,Culture,Migos,3.5K,300 Entertainment,Culture
10993,199,Night Visions,Imagine Dragons,3.5K,Interscope,Night Visions


## Download collection of simplified album objects from spotify

Get unique cleaned titles from dataframe

In [50]:
titles_list = list(rs_200_year_df.title_cleaned.unique())

inspect this

In [52]:
print(titles_list[:20])

['Indigo', 'When We All Fall Asleep, Where Do We Go?', "Let's Rock", '7 EP', 'Free Spirit', 'Perfect Ten', 'OASIS', 'Happiness Begins', 'Baby on Baby', 'Spider-Man: Into the Spider-Verse', 'thank u, next', "This One's For You Too", 'Acid Rap', 'Case Study 01', 'Beerbongs & Bentleys', 'Scorpion', 'Die a Legend', 'Cuz I Love You', 'Hoodie SZN', 'Father of Asahd']


In [56]:
print(len(titles_list))

611


Download spotify simplified album data for each of these titles

In [ ]:
if False: # make me true if needed
    searches, fails = gsad.search_for_albums(titles_list)

save to pickle

In [ ]:
if False: # make me true if needed
save_pickle(searches, project_dir + '/data/raw/spotipy_album_searches_rs_200_year.pickle')

load from pickle

In [53]:
if True: # make me true if needed
    searches = load_pickle(project_dir + '/data/raw/spotipy_album_searches_rs_200_year.pickle')

inspect searches

In [54]:
print(list(searches.keys())[:10])

['Indigo', 'When We All Fall Asleep, Where Do We Go?', "Let's Rock", '7 EP', 'Free Spirit', 'Perfect Ten', 'OASIS', 'Happiness Begins', 'Baby on Baby', 'Spider-Man: Into the Spider-Verse']


In [55]:
len(titles_list)

611

## Generate album title: album id dict

try to generate dictionary of album names to spotify album ids. Printed output indicates when an ID wasn't found. ie: when spotify returned a paging item with an empty list from the search

In [57]:
album_name_id_dict= gsad.get_album_name_id_dict(searches)

id not found for:        ?
error: list index out of range

id not found for:        A Winter Romance - Let It Snow! Let It Snow! Let It Snow!
error: list index out of range

id not found for:        The Very Best of Dean Martin
error: list index out of range

id not found for:        Season's Greetings From Perry Como
error: list index out of range

id not found for:        Dirty Sprite 2 (DS2)
error: list index out of range

id not found for:        The Spirit Of Radio: Greatest Hits 1974-1987
error: list index out of range

id not found for:        !
error: list index out of range

id not found for:        World War Joy...Call You Mine
error: list index out of range

id not found for:        World War Joy...Push My Luck
error: list index out of range

id not found for:        Charlie's Angels 2019 (Original Motion Picture Soundtrack)
error: list index out of range

id not found for:        Dean Martin Chante Noël
error: list index out of range



It appears from the printed output that 11 were missing. That's okay for now. Let's see the length of album_name_id_dict

In [58]:
len(album_name_id_dict)

600

That number makes sense! We had 611, lost 11, and now have 600.

## Download full album data for each of the 600 albums for which we have a spotify id

In [ ]:
if False: # make me true if needed
    dict_of_full_album_objects = gsad.get_dict_of_full_album_objects(album_name_id_dict)
    print('done!')

Save a pickle of it

In [61]:
directory = project_dir + '/data/raw/'
file_name = 'dict_of_full_spotify_album_objects_rs_200_year_2019.pickle'
if False: # make me true if needed
    save_pickle(dict_of_full_album_objects, directory + file_name)

Load it from a pickle, if needed

In [62]:
directory = project_dir + '/data/raw/'
file_name = 'dict_of_full_spotify_album_objects_rs_200_year_2019.pickle'
if True:
    dict_of_full_album_objects = load_pickle(directory + file_name)

inspect it

In [63]:
len(dict_of_full_album_objects)

600

In [65]:
print(list(dict_of_full_album_objects.keys())[:15])

['Indigo', 'When We All Fall Asleep, Where Do We Go?', "Let's Rock", '7 EP', 'Free Spirit', 'Perfect Ten', 'OASIS', 'Happiness Begins', 'Baby on Baby', 'Spider-Man: Into the Spider-Verse', 'thank u, next', "This One's For You Too", 'Acid Rap', 'Case Study 01', 'Beerbongs & Bentleys']


In [66]:
dict_of_full_album_objects['Let\'s Rock']

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7mnBLXK823vNxN3UWB7Gfz'},
   'href': 'https://api.spotify.com/v1/artists/7mnBLXK823vNxN3UWB7Gfz',
   'id': '7mnBLXK823vNxN3UWB7Gfz',
   'name': 'The Black Keys',
   'type': 'artist',
   'uri': 'spotify:artist:7mnBLXK823vNxN3UWB7Gfz'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'SA',
  'SE',
  'SG',
  'SK',
  'SV',
  'TH',
  'TN',
  'TR',
  'TW',
  'US',
  'UY',
  'VN',
  'ZA'],
 'copyrights': [{'

In [67]:
dict_of_full_album_objects['Let\'s Rock'].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [70]:
dict_of_full_album_objects['Let\'s Rock']['tracks']['items'][2]

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7mnBLXK823vNxN3UWB7Gfz'},
   'href': 'https://api.spotify.com/v1/artists/7mnBLXK823vNxN3UWB7Gfz',
   'id': '7mnBLXK823vNxN3UWB7Gfz',
   'name': 'The Black Keys',
   'type': 'artist',
   'uri': 'spotify:artist:7mnBLXK823vNxN3UWB7Gfz'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'SA',
  'SE',
  'SG',
  'SK',
  'SV',
  'TH',
  'TN',
  'TR',
  'TW',
  'US',
  'UY',
  'VN',
  'ZA'],
 'disc_number': 1,
 'duration_ms': 177768,

## Download track audio features for all tracks within the spotify full album objects

In [ ]:
if False: # make me True to download track audio features
    # load full spotify album objects
    file_path = str(project_dir) + '/data/raw/' + 'dict_of_full_spotify_album_objects_rs_200_year_2019.pickle'
    dict_of_full_spotify_album_objects = pickle_util.load_pickle(file_path)

    # using this, get track audio features
    dict_of_album_ids_to_list_of_track_audio_features = get_dict_of_album_ids_to_list_of_track_audio_features(dict_of_full_spotify_album_objects)
    file_path = str(project_dir) + '/data/raw/' + 'dict_of_album_ids_to_list_of_track_audio_features.pickle'
    pickle_util.save_pickle(dict_of_album_ids_to_list_of_track_audio_features, file_path)

In [2]:
if False: # make me True to download track audio features
    # load full spotify album objects
    file_path = str(project_dir) + '/data/raw/' + 'dict_of_full_spotify_album_objects_rs_200_year_2019.pickle'
    dict_of_full_spotify_album_objects = pickle_util.load_pickle(file_path)

    # using this, get track audio features
    dict_of_album_ids_to_list_of_track_audio_features = get_dict_of_album_ids_to_list_of_track_audio_features(dict_of_full_spotify_album_objects)
    file_path = str(project_dir) + '/data/raw/' + 'dict_of_album_ids_to_list_of_track_audio_features.pickle'
    pickle_util.save_pickle(dict_of_album_ids_to_list_of_track_audio_features, file_path)